In [2]:
import os
import torch
import torchvision
import torch.nn as nn
import torchtoolbox.transform as transforms
import torch.optim as optim
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from torchvision import models

In [3]:
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
class MyMobilenet (nn.Module):

    def __init__(self, mobilenet, num_class, freeze_conv=False, n_feat_conv=1280):

        super(MyMobilenet, self).__init__()
        
        self.features = nn.Sequential(*list(mobilenet.children())[:-1])

        if freeze_conv:
            for param in self.features.parameters():
                param.requires_grad = False

        self.classifier=nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=n_feat_conv, out_features=6, bias=True))   

    def forward(self, img):
        x = self.features(img)
        x = x.mean([2, 3])

        x = x.view(x.size(0), -1) # flatting
            
        return self.classifier(x)

In [5]:
def load_model (checkpoint_path, model, opt_fn=None, loss_fn=None, epoch=None):

    if not os.path.exists(checkpoint_path):
        raise Exception ("The {} does not exist!".format(checkpoint_path))

    ckpt = torch.load(checkpoint_path)
    model.load_state_dict(ckpt['model_state_dict'])

    if opt_fn is not None and loss_fn is not None:
        opt_fn.load_state_dict(ckpt['optimizer_state_dict'])
        epoch = ckpt['epoch']
        loss_fn = ckpt['loss']
        return model, opt_fn, loss_fn, epoch
    else:
        return model

In [6]:
model = MyMobilenet(models.mobilenet_v2(pretrained=True), 6)
model.to('cpu')
loss = nn.CrossEntropyLoss()
opt_fun = optim.Adam(model.parameters(), lr = 0.0001)
path = 'E:/Results/Multi_Class/Final_Model/best-checkpoint/best-checkpoint.pth' 

Modell = load_model(path, model,opt_fun, loss, epoch=33)
model.eval()

MyMobilenet(
  (features): Sequential(
    (0): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

In [7]:
labels = ['Atopic Dermatitis','Acne Vulgaris',  'Lichen Planus', 'Onychomycosis', 'Tinea Capitis', 'Unknown']

In [12]:
def predict(img):
    
    img = Image.open(img).convert('RGB')
    img = np.asarray(img)
    img = test_transform(img)
    img = torch.tensor(img, device=torch.device("cpu"))
    img = img[None,:,:,:]
    inputs = (img)
    
    preds = model(inputs)
    
    preds = torch.argmax(preds,1)
    
    #return preds
    print("The Skin Condition is {}".format(labels[preds.max()]))

In [14]:
img = 'E:/Results/Multi_Class/Dataset/test/AD37.jpg'
k = predict(img)

The Skin Condition is Lichen Planus


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
